In [1]:
import pandas as pd
from nltk.corpus import stopwords
from scrape_darklyrics import get_band_lyrics

In [26]:
import re

def strip_lyrics(lyrics):
    text = ' '.join(lyrics)
    words = re.split(' |,|\.', text)
    out = [word.lower() for word in words if word != '']
    return out

def word_count(a):
    words = []
    for word in a:
        if any(char.isalpha() for char in word):
            if '\'' in word:
                word = word[:word.index('\'')]
            word = word.replace('(', '')
            word = word.replace('(', '')
            words.append(word)
    keys = sorted(set(words) - set(stopwords.words('english')))
    return {key: words.count(key) for key in keys if '\'re' not in key}

def band_word_count(band_lyrics):
    df_list = []
    for album_lyrics in band_lyrics.values():
        for song, lyrics in album_lyrics.items():
            word_count_dict = word_count(strip_lyrics(lyrics))
            song_df = pd.DataFrame.from_dict(word_count_dict, orient='index')
            if len(song_df) != 0:
                song_name = song[song.index('.')+2:]
                song_df.columns = [song_name]
                df_list.append(song_df)
    df = pd.concat(df_list, axis=1, sort=True).fillna(0)
    return df

In [30]:
df = band_word_count(get_band_lyrics('betweentheburiedandme'))

In [33]:
df.sum(axis=1)

abandoned        1.0
abduction        1.0
abort            1.0
abound           1.0
above/below      1.0
abscess          2.0
absolutely       1.0
accept           5.0
achievement      1.0
acknowledge      1.0
across          10.0
act              4.0
action           1.0
actions          3.0
actual           3.0
add              1.0
admit            1.0
adult            2.0
adults           2.0
advantage        2.0
affair           1.0
affect           1.0
affection        2.0
afloat           1.0
afraid           5.0
after?           1.0
age-old          1.0
agenda           1.0
aggressive       1.0
ago              4.0
                ... 
wrist            1.0
write            1.0
writes:          1.0
writing          8.0
written          1.0
wrong            2.0
wrote            1.0
x                1.0
ya               2.0
yawning          1.0
yeah             3.0
yeah!!!          1.0
yeah!!!!         1.0
yeah!)           1.0
year            10.0
year:            1.0
years        